# Tools


- Author: [Jaeho Kim](https://github.com/Jae-hoya)
- Design: []()
- Peer Review:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-4/sub-graph.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239937-lesson-2-sub-graphs)

## OverView

A tool is an interface that allows agents, chains, or LLMs to interact with the external world.

LangChain provides built-in tools that are easy to use, and it also enables users to easily build custom tools.

**You can find the list of tools integrated into LangChain at the link below.**

- [List of Tools Integrated with LangChain](https://python.langchain.com/docs/integrations/tools/)

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Built-in tools](#built-in-tools)
- [Python REPL Tool](#python-repl-tool)
- [Search API Tool(Tavily)](#search-api-tooltavily)
- [Image Generation Tool (DALL-E)](#image-generation-tool-dall-e)
- [Custom Tools](#custom-tools)
### References

- [LangChain: List of Integrated Tools](https://python.langchain.com/docs/integrations/tools/)
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)
- [LangChain: PythonREPLTool](https://python.langchain.com/docs/integrations/tools/python/)
- [LangChain: Tavily Search](https://python.langchain.com/docs/integrations/tools/tavily_search/)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "langchain_experimental",
        "feedparser"
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "TAVILY_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "01-Tools",
    }
)

Environment variables have been set successfully.


Environment variables have been set successfully.
You can alternatively set API keys, such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [4]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

**Note**

When using tools, warning messages may be displayed.

For example, there are security warnings in the **REPL** environment.
`PythonREPLTool` is a tool for executing Python code and can potentially execute commands (e.g., file system access, network requests) that may pose security risks.
In such cases, LangChain or Python itself may output warning messages.

To ignore these warning messages, you can use the following code:

In [5]:
import warnings

# Ignore warning messages.
warnings.filterwarnings("ignore")

## Built-in tools

You can use pre-defined tools and toolkits provided by LangChain.

A tool refers to a single utility, while a toolkit combines multiple tools into a single unit for use.

You can find the relevant tools at the link below.

**Note**
- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

## Python REPL Tool

This tool provides a class for executing Python code in a **REPL (Read-Eval-Print Loop)** environment.
- [PythonREPLTool](https://python.langchain.com/docs/integrations/tools/python/)

**Description**

- Provides a Python shell environment.
- Executes valid Python commands as input.
- Use the `print(...)` function to view results.

**Key Features**

- sanitize_input: Option to sanitize input (default: True)
- python_repl: Instance of **PythonREPL** (default: executed in the global scope)

**Usage**

- Create an instance of `PythonREPLTool `.
- Execute Python code using the `run` , `arun` , or `invoke` methods.

**Input Sanitization**

- Removes unnecessary spaces, backticks, the keyword "python," and other extraneous elements from the input string.

In [6]:
from langchain_experimental.tools import PythonREPLTool

# Creates a tool for executing Python code.
python_tool = PythonREPLTool()

In [7]:
# Executes Python code and returns the results.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



Below is an example of requesting an LLM to write Python code and returning the results.

**Workflow Summary**
1. Request the LLM model to write Python code for a specific task.
2. Execute the generated code to obtain the results.
3. Output the results.

**Note**

I recommend using a model equivalent to or higher than GPT-4.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# A function that executes Python code, outputs intermediate steps, and returns the tool execution results.
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# A prompt requesting Python code to be written.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# Create LLM model.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Create a chain using the prompt and the LLM model.
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [9]:
# Outputting the results.
print(chain.invoke("Write code to generate Powerball numbers."))

CODE:
import random

def generate_powerball():
    main_numbers = sorted(random.sample(range(1, 70), 5))
    powerball = random.randint(1, 26)
    return main_numbers, powerball

print(generate_powerball())
([23, 27, 32, 34, 58], 3)



## Search API Tool(Tavily)

This is a tool that implements a search function using the `Tavily` Search API. It provides two main classes: `TavilySearchResults` and `TavilyAnswer` .

**API Key Issuance URL**
- https://app.tavily.com/

Set the issued API key as an environment variable.

For example, configure the .env file as follows:

```
TAVILY_API_KEY=tvly-abcdefghijklmnopqrstuvwxyz
```

### TavilySearchResults

**Description**
- Queries the Tavily Search API and returns results in JSON format.
- A search engine optimized for comprehensive, accurate, and reliable results.
- Useful for answering questions about current events.

**Key Parameters**
- `max_results` (int): Maximum number of search results to return (default: 5).
- `search_depth` (str): Search depth ("basic" or "advanced").
- `include_domains` (List[str]): List of domains to include in search results.
- `exclude_domains` (List[str]): List of domains to exclude from search results.
- `include_answer` (bool): Whether to include a short answer to the original query.
- `include_raw_content` (bool): Whether to include refined HTML content from each site.
- `include_images` (bool): Whether to include a list of images related to the query.

**Return Value**
- A JSON-formatted string containing the search results (url, content).

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create tool.
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io"],
    # exclude_domains = []
)

In [11]:
# Execute tool.
tool.invoke({"query": "What is Langchain Tools?"})

[{'url': 'https://stonefishy.github.io/2024/11/12/introduction-to-langchain-make-ai-smarter-and-easy-to-use/',
  'content': 'What is LangChain? LangChain is a tool for developers that helps them build applications using large language models (LLMs)—the same kind of AI that powers chatbots, writing assistants, and more. LLMs can understand and generate text in a way that sounds like a real person. However, using these models to make powerful apps can be complicated.'},
 {'url': 'https://kirenz.github.io/lab-langchain-functions/slides/06_functional_conversation.html',
  'content': "LangChain helps developers leverage the power of language models in their applications. Finished chain. {'input': 'what is langchain?', 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs)."},
 {'url': 'https://aws-samples.github.io/amazon-bedrock-samples/agents-and-function-calling/open-source-agents/langgraph/langgraph-single-agent/',
  'con

## Image Generation Tool (DALL-E)

- `DallEAPIWrapper Class` : A wrapper for OpenAI's DALL-E image generator.

This tool allows easy integration of the DALL-E API to implement text-based image generation functionality. With various configuration options, it can be utilized as a flexible and powerful image generation tool.

**Key Properties**

- `model` : The name of the DALL-E model to use ( **dall-e-2**, **dall-e-3** ).

- `n` : Number of images to generate (default: 1).

- `size` : Size of the generated image:
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style` : Style of the generated image ( **natural** , **vivid** ).

- `quality` : Quality of the generated image ( **standard**, **hd** ).

- `max_retries` : Maximum number of retries for generation.

**Key Features**
- Generates images based on text descriptions using the DALL-E API.

**Workflow Summary**

The following is an example of generating images using the `DALL-E` Image Generator.

This time, we will use the `DallEAPIWrapper` to generate images.

The input prompt will request the LLM model to write a prompt for generating images.

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# Define a prompt template for DALL-E image generation
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# Create a chain connecting the prompt, LLM, and output parser
chain = prompt | llm | StrOutputParser()

# Execute the chain
image_prompt = chain.invoke(
    {"image_desc": "A Neo-Classicism painting satirizing people looking at their smartphones."}
)

# Output the image prompt
print(image_prompt)

Create a Neo-Classical painting that humorously depicts a group of elegantly dressed individuals in a grand, classical setting, all intently staring at their smartphones instead of engaging with each other. The scene should feature opulent architecture, with marble columns, intricate frescoes, and lush drapery in the background. Characters should embody the fashion and demeanor of the Neo-Classical era: men in tailored coats with waistcoats and women in flowing gowns with empire waistlines. Their expressions should range from confusion to fascination as they interact with their devices. Include subtle anachronistic elements, like modern technology blending with classical motifs—perhaps a smartphone adorned with gold leaf or digital notifications emerging from a classic scroll. Capture the irony of the situation with a slightly exaggerated, satirical tone, and ensure the color palette is rich and vibrant, reflecting the artistry of the Neo-Classical period.


Let’s use the previously generated image prompt as input to the `DallEAPIWrapper` to generate an image.

In [ ]:
# Importing the DALL-E API Wrapper
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

dalle = DallEAPIWrapper(
    model="dall-e-3",
    size="1024x1024",
    quality="standard",
    n=1,
)

# query
query = "A Neo-Classicism painting satirizing people looking at their smartphones."


# Generate image and retrieve URL
# Use chain.invoke() to convert the image description into a DALL-E prompt
# Use dalle.run() to generate the actual image
image_url = dalle.run(chain.invoke({"image_desc": query}))

# Display the generated image.
Image(url=image_url, width=500)

## Custom Tools

In addition to the built-in tools provided by LangChain, you can define and use your own custom tools.

To do this, use the `@tool` decorator provided by the `langchain.tools` module to convert a function into a tool.

### @tool Decorator

This decorator allows you to transform a function into a tool. It provides various options to customize the behavior of the tool.

**How to Use**
1. Apply the `@tool` decorator above the function.
2. Set the decorator parameters as needed.

Using this decorator, you can easily convert regular Python functions into powerful tools, enabling automated documentation and flexible interface creation.

In [14]:
from langchain.tools import tool


# Convert a function into a tool using a decorator.
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [15]:
# Execute tool.
add_numbers.invoke({"a": 3, "b": 4})

7

In [16]:
# Execute tool.
multiply_numbers.invoke({"a": 3, "b": 4})

12

## Creating a Custom Tool for Google News Article Search

Define the `GoogleNews` class, which will be used as a tool to search for Google News articles.

**Note**
- No API key is required (because it uses RSS feeds).

This tool searches for news articles provided by **news.google.com** .

**Description**
- Uses the Google News search API to retrieve the latest news.
- Allows searching for news based on keywords.

**Key Parameters**
- `k` (int): Maximum number of search results to return (default: 5).

```python
# hl: Language, gl: Region, ceid: Region and Language Code
url = f"{self.base_url}?hl=en&gl=US&ceid=US:en" 
```

In the code, you can adjust the search results' language and region by modifying the language (hl), region (gl), and region and language code (ceid).

**Note**

Save the provided code as `google_news.py` , and then you can import it in other files using `from google_news import GoogleNews` .


In [24]:
#%pip install feedparser

In [19]:
import feedparser
from urllib.parse import quote
from typing import List, Dict, Optional


class GoogleNews:
    """
    This is a class for searching Google News and returning the results.
    """

    def __init__(self):
        """
        Initializes the GoogleNews class.
        Sets the base_url attribute.
        """
        self.base_url = "https://news.google.com/rss"

    def _fetch_news(self, url: str, k: int = 3) -> List[Dict[str, str]]:
        """
        Fetches news from the given URL.

        Args:
            url (str): The URL to fetch the news from.
            k (int): The maximum number of news articles to fetch (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing news titles and links.
        """
        news_data = feedparser.parse(url)
        return [
            {"title": entry.title, "link": entry.link}
            for entry in news_data.entries[:k]
        ]

    def _collect_news(self, news_list: List[Dict[str, str]]) -> List[Dict[str, str]]:
        """
        Formats and returns the list of news articles.

        Args:
            news_list (List[Dict[str, str]]): A list of dictionaries containing news information.

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        if not news_list:
            print("No news available for the given keyword.")
            return []

        result = []
        for news in news_list:
            result.append({"url": news["link"], "content": news["title"]})

        return result

    def search_latest(self, k: int = 3) -> List[Dict[str, str]]:
        """
        Searches for the latest news.

        Args:
            k (int): The maximum number of news articles to search for (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        #url = f"{self.base_url}?hl=ko&gl=KR&ceid=KR:ko"
        url = f"{self.base_url}?hl=en&gl=US&ceid=US:en" # hl: 언어, gl: 지역, ceid: 지역 및 언어 코드
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)

    def search_by_keyword(
        self, keyword: Optional[str] = None, k: int = 3
    ) -> List[Dict[str, str]]:
        """
        Searches for news using a keyword.  

        Args:
            keyword (Optional[str]): The keyword to search for (default: None).
            k (int): The maximum number of news articles to search for (default: 3).

        Returns:
            List[Dict[str, str]]: A list of dictionaries containing URLs and content.
        """
        if keyword:
            encoded_keyword = quote(keyword)
            url = f"{self.base_url}/search?q={encoded_keyword}&hl=en&gl=US&ceid=US:en"
        else:
            url = f"{self.base_url}?hl=en&gl=US&ceid=US:en"
        news_list = self._fetch_news(url, k)
        return self._collect_news(news_list)


In [20]:
google_tool = GoogleNews()

In [21]:
google_tool.search_by_keyword("AI Investment")

[{'url': 'https://news.google.com/rss/articles/CBMimAFBVV95cUxNaThyMnBjNjJpQjlQQTcwQjdoTEZPc1plbDdYU29YWlNVakE1NzVkV2ZNZHRyYlFSSjg1VDRNX2ZPT1NZdkM0ckVxMmxwaW1PbTdpay1EX2lUbFNqblIxOXdUZ3VDTVZoVmRlWVZXLTBUNG5SSUJZa3RieEFST0VjQ1hSRkxWNzRXSlBYb3k3QzEzUGdtNHo2dg?oc=5',
  'content': "A Once-in-a-Decade Investment Opportunity: 1 Artificial Intelligence (AI) Semiconductor Stock to Buy Hand Over Fist and Hold for the Next 10 Years (Hint: It's Not Nvidia) - The Motley Fool"},
 {'url': 'https://news.google.com/rss/articles/CBMixAFBVV95cUxNZmZLZHlKQXZCZllQTTNTcEt4Vy1GZDhXNFdlbnZURXY3UzJCTWNneDZja1pwa003LW45V2NFcjR1RzZOdk5PcnBUV3ZsemV2eTlHZUlMdXJHTHBsNzBVaHJwRmNIR0JtQzhaNU1IODE5WlJKZk5HV0xxVThUZjhma2JnbUFVUEktRFRxcFNrS1B0WDV4ZTZTdzVIQWVCakwzM1VUd0NhaGozZlVod2NkZW5qay1QWEgxcnpNT2lkNGRoN1Jy0gHKAUFVX3lxTE5TS2I1RjRmYi1nUDFNQU9jckRxLWdPRC1kbUxuUVQ5dnFLQWN3OW5zdXRQOW5xZmtSeHZ1UTdNN1ZWMUdrSGdQU0ZtV0lFM2lfZjhyWk8wSHVmVmpqVGtUSXo1TDh4R3dmWnpBSFhEMjI1WklGa1BMc3VnTWgtUEc5NlkyTUZqZ3M0V3o3YmwzX3dqVDk5clU5dTFOOF

In [22]:
from langchain.tools import tool
from typing import List, Dict


# Create a tool for searching news by keyword
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [23]:
# Execution Results
search_keyword.invoke({"query": "LangChain AI"})

LangChain AI


[{'url': 'https://news.google.com/rss/articles/CBMib0FVX3lxTE9qeUFQd08yN3pXenFYeWRTM1dHN3ZtTGVvaDAzREVNRU02ZVhNRFZCWmFpRm9obTN5QXF1TlpaYUI3d2VJYTBNeHNyUGJ3MXdVQWNDVW5MTVk4UDlfazFfX1hRRWpUeHBfUGRfS0pBdw?oc=5',
  'content': 'Vulnerabilities in LangChain Gen AI - Unit 42'},
 {'url': 'https://news.google.com/rss/articles/CBMixgFBVV95cUxNUzhFZEtsS2FrNTZ1ejVDejBQRThtSDA5TUtMbWRHOUxfUEtBVl8teERpZXp5U2wzbktZeWdTWnhKc2V4RnM2SXc0TzBsNVlQMktqUXRwNDliQW9yaTVnT3lzZ0lxeThobTliUEJRd05VM3V1anJ3bUh4cGJRbk44a1V0MVMzSEZfc2c4am04QnJBM2ZhZThTQ1NvYlhQUVBZbmhJWVZQdzlWWjE0UVdQRjYtWU1iSWl6WFBBTXdsQ2ROSHUxWmc?oc=5',
  'content': 'LangChain Meets Home Assistant: Unlock the Power of Generative AI in Your Smart Home - Towards Data Science'},
 {'url': 'https://news.google.com/rss/articles/CBMi6wFBVV95cUxPNEtmajI2MWxEb2FRSUNLWkhBQmV5VThPUVllNkRzbGEwOW1mRDZVZEhxYnI3X3BxZUZ1WThGbmF6WUV6MnVSaDJtN0ZUd2VZdGt5NV9CNHJRVXllVElnZE5DaVJDdFBSZHU5aGI3XzM3RTR1YVdyU19pYjJER2NTM080dlYyVjd6MDNFOTFudm5WRG1zdk9WNUxJTVpoakxEWlZxR3RaSm